In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from gensim.models import Word2Vec
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [25]:
df_test = pd.read_csv("../data/test/test.csv")
df_train = pd.read_csv("../data/train/train.csv")
dff = pd.read_csv("../data/test.csv")

In [26]:
df_test.head()

,doc_id,titre,type,difficulte,cout,ingredients,recette
0,recette_220079.xml,Gratin de brocoli et lardons,Plat principal,Facile,Bon marché,- 1 brocoli - 100 g de lardons - 3 œufs - 20 c...,Préchauffer le four à 170°C (thermostat 5).Fai...
1,recette_25691.xml,Langoustines à la crème,Entrée,Facile,Moyen,"- 24 langoustines - 1,5 l d'eau - 50 cl de c...","Dans une cocotte, verser l'eau, les oignons et..."
2,recette_15272.xml,Filet de dinde aux asperges,Plat principal,Très facile,Bon marché,- 1 gros filet de dinde - 1 bocal de petites a...,Couper le filet de dinde en gros morceau (comm...
3,recette_55064.xml,Gâteau marandais,Dessert,Très facile,Bon marché,- 250 g de farine - 250 g de sucre - 125 g de ...,"Mélanger le sucre, l'angélique, le beurre fond..."
4,recette_63116.xml,Tarte à l'artichaut,Entrée,Facile,Bon marché,- 2 artichauts de taille moyenne - 1 pâte bris...,Cuire les artichauts dans l'eau bouillante pen...


In [27]:
df_train.head()

,doc_id,titre,type,difficulte,cout,ingredients,recette
0,recette_19924.xml,Bûche de Noël pistachée,Dessert,Moyennement difficile,Moyen,- Pour le biscuit : - 4 œufs + 2 jaunes - 110 ...,Préchauffez le four à (210°C). Étalez du papie...
1,recette_25149.xml,Tarte aux pommes au fromage blanc,Dessert,Facile,Moyen,- 1 rouleau de pâte feuilletée ou brisée - 3 p...,"Pelez et coupez les pommes en quartiers, retir..."
2,recette_40068.xml,Salade de chèvre chaud au miel et pesto,Entrée,Très facile,Bon marché,- 4 chèvres frais - 4 poignées de roquette (ou...,Faire une couronne avec la roquette dans chaqu...
3,recette_62942.xml,Pommes de terre farcies aux escargots,Entrée,Très facile,Moyen,- 2 très grosses pommes de terre - 4 douzaines...,Faire cuire les pommes de terre à l'eau mais p...
4,recette_12714.xml,Tarte au fromage et à l'abricot,Dessert,Facile,Bon marché,- Pour la pâte : - 200 g de farine - 1 oeuf - ...,Coupez en petit dés le beurre que vous mélange...


In [28]:
X_train = df_train['recette']
X_test = df_test['recette']

Y_train = df_train['type']
Y_test = df_test['type']

dff_X = dff["recette"]
dff_Y = dff["type"]

In [29]:
stop_words = set(stopwords.words('french'))
def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [30]:
X_train = X_train.apply(preprocess)
X_test = X_test.apply(preprocess)
dff_X = dff_X.apply(preprocess)

In [31]:
sentences = [sentence.split() for sentence in X_train]
w2v_model = Word2Vec(sentences, workers=4)

In [32]:
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train = np.array([vectorize(sentence) for sentence in X_train])
X_test = np.array([vectorize(sentence) for sentence in X_test])
dff_X = np.array([vectorize(sentence) for sentence in dff_X])

In [33]:
clf = LogisticRegression(max_iter=2000)
clf.fit(X_train, Y_train)

LogisticRegression(max_iter=2000)

In [34]:
y_pred = clf.predict(X_test)
test_pred = clf.predict(dff_X)
print('Accuracy:', accuracy_score(Y_test, y_pred))
print('Precision:', precision_score(Y_test, y_pred, average='macro'))
print('Recall:', recall_score(Y_test, y_pred, average='macro'))
print('F1 score:', f1_score(Y_test, y_pred,  average='macro'))
print('F1 score:', f1_score(dff_Y, test_pred,  average='macro'))

Accuracy: 0.8333333333333334
Precision: 0.8245731012739453
Recall: 0.8002525772193524
F1 score: 0.8072703382096069
F1 score: 0.8088404971162769


In [35]:
values, counts = np.unique(df_train['type'], return_counts=True)
maxval = values[np.argmax(counts)]
print(maxval)
majoritaire = np.ones_like(np.chararray((len(maxval),len(df_train))))
majoritaire[:,] = maxval
print(majoritaire)

Plat principal
[[b'P' b'P' b'P' ... b'P' b'P' b'P']
 [b'P' b'P' b'P' ... b'P' b'P' b'P']
 [b'P' b'P' b'P' ... b'P' b'P' b'P']
 ...
 [b'P' b'P' b'P' ... b'P' b'P' b'P']
 [b'P' b'P' b'P' ... b'P' b'P' b'P']
 [b'P' b'P' b'P' ... b'P' b'P' b'P']]
